In [61]:
import pandas as pd
from unidecode import unidecode
from difflib import get_close_matches

In [62]:
# --- Cargar datos  ---
tabla1 = pd.read_csv("C:/Users/juana/Desktop/liquideuda_project/data_colegio/EPC2.csv")
tabla2 = pd.read_csv("C:/Users/juana/Downloads/ID GEO ESPAÑA - Hoja 1.csv")

In [63]:
import re
from pathlib import Path

# --- 1) Cargar tabla (EPC2) ---
path = Path(r"C:\Users\juana\Desktop\liquideuda_project\data_colegio\EPC2.csv")
tabla1 = pd.read_csv(path, encoding="utf-8-sig")

# --- 2) Función para limpiar/normalizar nombre de provincia ---
def clean_prov(name):
    if pd.isna(name):
        return ""
    s = str(name)
    # quitar códigos numéricos al inicio: "02 Albacete" -> "Albacete"
    s = re.sub(r"^\s*\d+\s*", "", s)
    # quitar paréntesis y puntos
    s = s.replace("(", " ").replace(")", " ").replace(".", " ")
    # pasar a ascii, minúsculas, quitar acentos
    s = unidecode(s).lower()
    # reemplazar separadores por espacios
    s = re.sub(r"[,/\\\-]", " ", s)
    # eliminar multiples espacios y strip
    s = re.sub(r"\s+", " ", s).strip()
    return s

# crear columna temporal con provincia limpia
tabla1["prov_limpia"] = tabla1["Comunidad/Provincia"].apply(clean_prov)

# --- 3) Lista manual provincia -> comunidad (valores en el estándar) ---
# Observación: los nombres de las CC.AA los dejamos en el estándar que ya defini.
manual_pairs = [
    # Andalucia
    ("almeria","Andalucia"), ("cadiz","Andalucia"), ("cordoba","Andalucia"), ("granada","Andalucia"),
    ("huelva","Andalucia"), ("jaen","Andalucia"), ("malaga","Andalucia"), ("sevilla","Andalucia"),
    # Aragon
    ("huesca","Aragon"), ("teruel","Aragon"), ("zaragoza","Aragon"),
    # Asturias
    ("asturias","Asturias"),
    # Baleares
    ("balears illes","Baleares"), ("baleares","Baleares"),
    # Canarias
    ("palmas las","Canarias"), ("las palmas","Canarias"), ("santa cruz de tenerife","Canarias"),("Las Palmas de Gran Canaria","Canarias"),
    # Cantabria
    ("cantabria","Cantabria"),
    # Castilla y Leon
    ("avila","Castilla y Leon"), ("burgos","Castilla y Leon"), ("leon","Castilla y Leon"),
    ("palencia","Castilla y Leon"), ("salamanca","Castilla y Leon"), ("segovia","Castilla y Leon"),
    ("soria","Castilla y Leon"), ("valladolid","Castilla y Leon"), ("zamora","Castilla y Leon"),
    # Castilla La Mancha
    ("albacete","Castilla La Mancha"), ("ciudad real","Castilla La Mancha"), ("cuenca","Castilla La Mancha"),
    ("guadalajara","Castilla La Mancha"), ("toledo","Castilla La Mancha"),
    # Cataluña
    ("barcelona","Cataluña"), ("girona","Cataluña"), ("lleida","Cataluña"), ("tarragona","Cataluña"),
    # Valencia (Comunitat Valenciana)
    ("alicante alacant","Valencia"), ("alicante","Valencia"),
    ("castellon castelló","Valencia"), ("castellon","Valencia"),
    ("valencia valencia","Valencia"), ("valencia","Valencia"),
    # Extremadura
    ("badajoz","Extremadura"), ("caceres","Extremadura"),
    # Galicia
    ("coruna a","Galicia"), ("a coruna","Galicia"), ("lugo","Galicia"), ("ourense","Galicia"), ("pontevedra","Galicia"),
    # Madrid
    ("madrid","Madrid"),
    # Murcia
    ("murcia","Murcia"),
    # Navarra
    ("navarra","Navarra"),
    # Pais Vasco
    ("araba alava","Pais Vasco"), ("alava","Pais Vasco"), ("bizkaia","Pais Vasco"), ("gipuzkoa","Pais Vasco"),
    # La Rioja
    ("rioja la","La Rioja"), ("la rioja","La Rioja"),
    # Ceuta y Melilla
    ("ceuta","Ceuta"), ("melilla","Melilla"),
    # Total Nacional (opcional)
    ("total nacional","España"),
]

# --- 4) Construir diccionario con claves limpias ---
provincia_to_ccaa = {k: v for k, v in manual_pairs}

# --- 5) Mapear creando la columna comunidad_autonoma ---
# la clave que usamos es la versión limpia "prov_limpia"
tabla1["comunidad_autonoma_manual"] = tabla1["prov_limpia"].map(provincia_to_ccaa)

# --- 6) Verificar filas que no se asignaron (NaN) ---
no_asignadas = tabla1[tabla1["comunidad_autonoma_manual"].isna()]

print("Número de filas sin asignación manual:", len(no_asignadas))
print("\nProvincias únicas sin asignar (muestra):")
print(no_asignadas["Comunidad/Provincia"].unique()[:50])

# Contar por valor original de provincia para ver qué nombres faltan
print("\nConteo de provincias no asignadas (top 30):")
print(no_asignadas["Comunidad/Provincia"].value_counts().head(30))

# --- 7) (Opcional) Mostrar ayuda para depurar: ver las claves limpias que no están en el diccionario ---
# claves únicas limpias en tabla vs dicc
claves_en_tabla = set(tabla1["prov_limpia"].unique())
claves_dicc = set(provincia_to_ccaa.keys())
faltantes = sorted(list(claves_en_tabla - claves_dicc))
print("\nClaves limpias presentes en tabla pero NO en diccionario (muestra 50):")
print(faltantes[:50])



Número de filas sin asignación manual: 432

Provincias únicas sin asignar (muestra):
['Andalucía' 'Aragón' 'Canarias' 'Las Palmas de Gran Canaria'
 'Castilla y León' 'Castilla-La Mancha' 'Cataluña' 'Comunitat Valenciana'
 'Extremadura' 'Galicia' 'Pais Vasco' 'Ceuta y Melilla']

Conteo de provincias no asignadas (top 30):
Comunidad/Provincia
Andalucía                     36
Aragón                        36
Canarias                      36
Las Palmas de Gran Canaria    36
Castilla y León               36
Castilla-La Mancha            36
Cataluña                      36
Comunitat Valenciana          36
Extremadura                   36
Galicia                       36
Pais Vasco                    36
Ceuta y Melilla               36
Name: count, dtype: int64

Claves limpias presentes en tabla pero NO en diccionario (muestra 50):
['andalucia', 'aragon', 'canarias', 'castilla la mancha', 'castilla y leon', 'cataluna', 'ceuta y melilla', 'comunitat valenciana', 'extremadura', 'galicia', 'las 

In [64]:

# --- Preprocesar textos: quitar acentos y pasar todo a minúsculas ---
def limpiar(texto):
    return unidecode(str(texto)).lower().strip()

tabla1["Comunidad/Provincia_limpia"] = tabla1["Comunidad/Provincia"].apply(limpiar)
tabla2["Provincia_limpia"] = tabla2["Provincia"].apply(limpiar)
tabla2["CA_limpia"] = tabla2["Comunidad Autónoma"].apply(limpiar)

# --- Diccionarios auxiliares ---
dicc_provincias = dict(zip(tabla2["Provincia_limpia"], tabla2["ID GEO"]))
dicc_provincia_a_ca = dict(zip(tabla2["Provincia_limpia"], tabla2["CA_limpia"]))
dicc_ca = {v: k for k, v in zip(tabla2["CA_limpia"], tabla2["Comunidad Autónoma"])}  # limpio→original

# --- Función que busca coincidencia ---
def obtener_valor(valor):
    # 1) Intentamos buscar provincia
    posibles = get_close_matches(valor, dicc_provincias.keys(), n=1, cutoff=0.8)
    if posibles:
        return dicc_provincias[posibles[0]]  # Devuelve ID GEO

    # 2) Si no es provincia, probamos si es CA
    posibles_ca = get_close_matches(valor, dicc_ca.keys(), n=1, cutoff=0.8)
    if posibles_ca:
        return dicc_ca[posibles_ca[0]]  # Devuelve nombre de la CA (original con tildes)

    # 3) Si no coincide con nada, devolvemos "DESCONOCIDO"
    return "DESCONOCIDO"

# --- Crear nueva columna al inicio ---
tabla1.insert(0, "ID GEO / CA", tabla1["Comunidad/Provincia_limpia"].apply(obtener_valor))

# --- Limpiar columnas auxiliares ---
tabla1 = tabla1.drop(columns=["Comunidad/Provincia_limpia"])

# --- Resultado ---
print(tabla1.head())
df=tabla1


# Función para limpiar nombres de columnas
def limpiar_columna(col):
    col = unidecode(col)             # quita acentos
    col = col.lower()                # pasa a minúsculas
    col = col.replace(" ", "_")      # espacios por _
    col = col.replace("/", "_")      # / por _
    col = col.replace("-", "_")      # - por _
    col = ''.join(c for c in col if c.isalnum() or c == "_")  # solo letras/números/_
    return col

# Aplicar a todas las columnas
df.columns = [limpiar_columna(c) for c in df.columns]


# Eliminar columnas que tengan todos los valores vacíos (NaN o "")
df = df.dropna(axis=1, how="all")           # elimina si todo es NaN
df = df.loc[:, (df != "").any(axis=0)]      # elimina si todo es cadena vacía

print("Columnas después de limpiar:", df.columns.tolist())


# Eliminar la columna de índice si se importó como parte del DataFrame
df = df.reset_index(drop=True)

# Eliminar columna llamada 'unnamed_0' si existe
df = df.drop(columns=['unnamed_0'], errors='ignore')
df
# Lista de columnas a convertir a numerico
cols_a_convertir = ["total","t_voluntario","t_necesario","t_consecutivo","c_ordinario","c_especial","c_sin_masa"]

# Convertir a numérico, los errores se convierten en NaN
for col in cols_a_convertir:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print("Columnas convertidas a numéricas correctamente.")


# Cantidad de vacíos y porcentaje por columna
vacios = df.isna().sum()  # cuenta NaN por columna
porcentaje = (vacios / len(df) * 100).round(2)  # calcula el % respecto al total de filas

# Crear un DataFrame resumido
resumen_vacios = pd.DataFrame({
    "Columnas": df.columns,
    "Cantidad vacíos": vacios.values,
    "Porcentaje vacíos": porcentaje.values
})

print(resumen_vacios)

# Filtrar filas donde id_geo sea "DESCONOCIDO" (ignorando mayúsculas/minúsculas)
desconocidos = df[df['id_geo___ca'].str.upper() == 'DESCONOCIDO']

# Contar la cantidad de entradas por provincia dentro de desconocidos
resumen_provincias = desconocidos['comunidad_provincia'].value_counts().sort_values(ascending=False)

print(resumen_provincias)



# Encuentra el índice de las filas a eliminar
indices = df[df['id_geo___ca'] == 'DESCONOCIDO'].index

# Elimina esas filas
df.drop(indices, inplace=True)

# Cambiar nombre de la columna
df = df.rename(columns={"filename": "periodo"})

# Quitar todo lo que hay después del punto (.) en cada valor de la columna Periodo
df["periodo"] = df["periodo"].str.split(".").str[0]

df[["año", "trimestre"]] = df["periodo"].str.split("_", expand=True)
df["trimestre"] = df["trimestre"].str.extract(r'(\d+)')  # Extrae solo el número


df = df.drop(columns=["periodo"])
df['id_geo___ca'].value_counts()
import os



   ID GEO / CA  Unnamed: 0      PJ Comunidad/Provincia Total T_Voluntario  \
0  DESCONOCIDO           0  fisica           Andalucía   199           22   
1          104           1  fisica             Almería    14            1   
2          111           2  fisica               Cádiz    27            4   
3          114           3  fisica             Córdoba    15            5   
4          118           4  fisica             Granada    17            1   

  T_Necesario T_Consecutivo  Unnamed: 6 C_Ordinario C_Especial C_Sin Masa  \
0           0           177         NaN         177          4         18   
1           0            13         NaN          13          0          1   
2           0            23         NaN          23          1          3   
3           0            10         NaN          11          0          4   
4           0            16         NaN          14          1          2   

       Filename prov_limpia comunidad_autonoma_manual  
0  2021_1T.xlsx   

In [65]:
df.columns

Index(['id_geo___ca', 'pj', 'comunidad_provincia', 'total', 't_voluntario',
       't_necesario', 't_consecutivo', 'c_ordinario', 'c_especial',
       'c_sin_masa', 'prov_limpia', 'comunidad_autonoma_manual', 'año',
       'trimestre'],
      dtype='object')

In [66]:
# 1. Eliminar la columna comunidad_provincia original
df = df.drop(columns=["comunidad_provincia"], errors="ignore")

# 2. Renombrar prov_limpia a comunidad_provincia
df = df.rename(columns={"prov_limpia": "comunidad_provincia"})

# 3. Renombrar comunidad_autonoma_manual a comunidad_autonoma
df = df.rename(columns={"comunidad_autonoma_manual": "comunidad_autonoma"})

# --- Verificar resultado ---
print("Columnas finales:", df.columns.tolist())
print(df.head())


Columnas finales: ['id_geo___ca', 'pj', 'total', 't_voluntario', 't_necesario', 't_consecutivo', 'c_ordinario', 'c_especial', 'c_sin_masa', 'comunidad_provincia', 'comunidad_autonoma', 'año', 'trimestre']
  id_geo___ca      pj  total  t_voluntario  t_necesario  t_consecutivo  \
1         104  fisica   14.0           1.0          0.0           13.0   
2         111  fisica   27.0           4.0          0.0           23.0   
3         114  fisica   15.0           5.0          0.0           10.0   
4         118  fisica   17.0           1.0          0.0           16.0   
5         121  fisica    8.0           1.0          0.0            7.0   

   c_ordinario  c_especial  c_sin_masa comunidad_provincia comunidad_autonoma  \
1         13.0         0.0         1.0             almeria          Andalucia   
2         23.0         1.0         3.0               cadiz          Andalucia   
3         11.0         0.0         4.0             cordoba          Andalucia   
4         14.0         1.0

In [67]:
# Filtrar solo las entradas NO numéricas
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]

# Revisar valores únicos
print(df_non_numeric['id_geo___ca'].value_counts())


id_geo___ca
castilla y leon         36
castilla-la mancha      36
comunitat valenciana    36
extremadura             36
galicia                 36
Name: count, dtype: int64


In [68]:
import os


# Filtrar solo las entradas NO numéricas
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]

# Revisar valores únicos
print(df_non_numeric['id_geo___ca'].value_counts())

df = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').notna()]
df_non_numeric = df[pd.to_numeric(df['id_geo___ca'], errors='coerce').isna()]
print(df_non_numeric['id_geo___ca'].value_counts())

id_geo___ca
castilla y leon         36
castilla-la mancha      36
comunitat valenciana    36
extremadura             36
galicia                 36
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


In [69]:


# Definir ruta destino
ruta_destino = r"C:\Users\juana\Desktop\liquideuda_project\datasets_clean"

# Guardar CSV en la ruta destino
ruta_csv = os.path.join(ruta_destino, "EPC2_clean.csv")
df.to_csv(ruta_csv, index=False, encoding="utf-8-sig")

print(f"Archivo guardado en: {ruta_csv}")


Archivo guardado en: C:\Users\juana\Desktop\liquideuda_project\datasets_clean\EPC2_clean.csv
